In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from bs4 import BeautifulSoup
import requests
import sqlite3

In [2]:
url = 'https://web.archive.org/web/20230902185326/https://en.wikipedia.org/wiki/List_of_countries_by_GDP_%28nominal%29'

In [3]:
db_name = "world_economies"
table_name = "countries_by_gdp"
csv_file = "countries_by_gdp.csv"
log_fie = "etl_logs.txt"
columns = ["country", "gdp_usd_million"]

In [4]:
def extract(url:str, table_attributes:list) -> pd.DataFrame:
    df = pd.DataFrame(columns=table_attributes)
    page = requests.get(url).text
    data = BeautifulSoup(page, "html.parser")
    table = data.find_all("tbody")[2]
    rows = table.find_all("tr")
    for row in rows:
        col = row.find_all("td")
        if len(col) == 0 or not col[0].find("a") or col[2].text == '—':
            continue
        country = col[0].find("a").text
        gdp = col[2].text
        country_dict = {
            "country": country,
            "gdp_billion": gdp
        }
        df1 = pd.DataFrame(country_dict, index=[0])
        df = pd.concat([df, df1], ignore_index=True)
    return df

In [5]:
def transform(df:pd.DataFrame) -> pd.DataFrame:
    df["gdp_billion"] = df["gdp_billion"].str.replace(",", "").astype(float)
    df["gdp_billion"] = np.round((df["gdp_billion"] / 1000), 2)
    df.rename(columns={"gdp_billion": "gdp_usd_billion"}, inplace=True)
    return df

In [6]:
def load_to_csv(df:pd.DataFrame, csv_file:str) -> None:
    df.to_csv(csv_file, index=False)

In [7]:
def load_to_sql(df:pd.DataFrame, db_name:str, table_name:str) -> None:
    conn = sqlite3.connect(db_name)
    df.to_sql(table_name, conn, if_exists="replace", index=False)
    conn.close()

In [8]:
def log(message:str) -> None:
    time_format = "%Y-%m-%d-%H:%M:%S"
    time_now = datetime.now()
    timestamp = time_now.strftime(time_format)
    with open(log_fie, "a") as f:
        f.write(f"{timestamp} ,{message} \n")

In [9]:
def run_query(query:str, sql_connection:sqlite3.Connection) -> pd.DataFrame:
    return pd.read_sql(query, sql_connection)

In [10]:
def main():
    log("ETL started")

    log("start extracting")
    extracted_data = extract(url, columns)
    log("end extracting")

    log("start tranfroming")
    transformed_data = transform(extracted_data)
    log("end tranfroming")

    log("start loading")
    log("load to csv")
    load_to_csv(transformed_data, csv_file)
    log("end loading to csv")
    log("load to sql")
    load_to_sql(transformed_data, db_name, table_name)
    log("end loading to sql")

    log("ETL ended")

In [11]:
main()

In [12]:
query = f"SELECT * FROM {table_name}"
sql_connection = sqlite3.connect(db_name)
result = run_query(query, sql_connection)
result

,country,gdp_usd_million,gdp_usd_billion
0,United States,None,26854.60
1,China,None,19373.59
2,Japan,None,4409.74
3,Germany,None,4308.85
4,India,None,3736.88
...,...,...,...
186,Marshall Islands,None,0.29
187,Palau,None,0.26
188,Kiribati,None,0.25
189,Nauru,None,0.15


In [13]:
%%bash
cat etl_logs.txt

2025-02-15-06:03:45 ,ETL started 
2025-02-15-06:03:45 ,start extracting 
2025-02-15-06:03:50 ,end extracting 
2025-02-15-06:03:50 ,start tranfroming 
2025-02-15-06:03:50 ,end tranfroming 
2025-02-15-06:03:50 ,start loading 
2025-02-15-06:03:50 ,load to csv 
2025-02-15-06:03:50 ,end loading to csv 
2025-02-15-06:03:50 ,load to sql 
2025-02-15-06:03:50 ,end loading to sql 
2025-02-15-06:03:50 ,ETL ended 


In [14]:
%%bash
head countries_by_gdp.csv

country,gdp_usd_million,gdp_usd_billion
United States,,26854.6
China,,19373.59
Japan,,4409.74
Germany,,4308.85
India,,3736.88
United Kingdom,,3158.94
France,,2923.49
Italy,,2169.74
Canada,,2089.67
